In [1]:
import sagemaker
from sagemaker.sklearn.model import SKLearnModel
from sagemaker.serializers import CSVSerializer
from sagemaker.model_monitor import DataCaptureConfig
import boto3
from sagemaker import get_execution_role, session, Session

In [2]:
model = SKLearnModel(model_data='s3://sagemaker-us-east-1-484322141311/sagemaker-scikit-learn-2023-04-08-01-55-03-087/output/model.tar.gz',
                    entry_point='entry_point.py',
                    role='Labrole',
                    framework_version='1.0-1')

In [ ]:
s3_capture_upload_path = 's3://final-dollar-semestre3-sa/dollar/inferences/'
endpoint_name = 'dollar-endpoint'
data_capture_config = DataCaptureConfig(
    enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path
)

predictor = model.deploy(instance_type='ml.m4.xlarge', 
                        initial_instance_count=1,
                        serializer=CSVSerializer(),
                        endpoint_name=endpoint_name,
                        data_capture_config=data_capture_config)

In [ ]:
predictor.predict([[738615]])

In [ ]:
session = Session()
payload = '738615'
response = session.sagemaker_runtime_client.invoke_endpoint(
                EndpointName=endpoint_name,
                ContentType="text/csv",
                Body=payload,
                InferenceId='1',  # unique ID per row
            )["Body"].read()

In [ ]:
response

In [ ]:
#Otra prueba

In [ ]:
train_channel = sagemaker.TrainingInput('s3://final-dollar-semestre3-sa/dollar/train', content_type='csv')
val_channel = sagemaker.TrainingInput('s3://final-dollar-semestre3-sa/dollar/test', content_type='csv')

In [ ]:
tuner.fit(
    {'train': train_channel, 'test': val_channel}
)